## 3.3 线性回归的简洁实现

- `torch.utils.data`模块提供有关数据处理的工具
- `torch.nn`模块定义了大量神经网络的层
- `torch.nn.init`模块定义了各种初始化方法
- `torch.optim`模块提供了很多常用的优化算法

### 3.3.1 生成数据集

生成与上一节中相同的数据集，其中`features`是训练数据特征，`labels`是标签。

In [1]:
import random
import torch
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2

features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float32)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float32)
print('features.shape:', features.shape)
print('labels.shape', labels.shape)

features.shape: torch.Size([1000, 2])
labels.shape torch.Size([1000])


### 3.3.2 读取数据

`pytorch`提供了`data`包来读取数据。由于`data`常用作变量名，将导入的`data`模块用`Data`代替。在每一次迭代中，将随机读取包含16个数据样本的小批量。

In [3]:
import torch.utils.data as Data

batch_size = 16
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [4]:
# 读取并打印第一个小批量数据样本
for X, y in data_iter:
    print(X, y)
    break

tensor([[-2.0501, -1.4243],
        [-0.0511,  0.0108],
        [-2.1359,  1.5977],
        [ 0.6027, -0.6070],
        [-0.6638,  0.6214],
        [ 1.6283,  0.6497],
        [-0.2126, -0.6961],
        [-1.0930, -1.2886],
        [ 0.5440,  1.2210],
        [-1.2386,  0.6222],
        [ 0.0883, -0.3734],
        [ 0.6561, -0.8581],
        [ 0.6516, -0.5971],
        [ 0.6519,  0.9336],
        [ 1.2895,  0.2487],
        [-0.0392,  0.3690]]) tensor([ 4.9475,  4.0756, -5.5011,  7.4769,  0.7565,  5.2370,  6.1519,  6.3990,
         1.1490, -0.3926,  5.6492,  8.4212,  7.5537,  2.3510,  5.9233,  2.8781])


### 3.3.3 定义模型

`PyTorch`提供了大量预定义的层，这是我们只需关注使用哪些层来构造模型。

首先，导入`torch.nn`模块。该模块定义了大量神经网络的层。`nn`的核心数据结构是`Module`，是一个抽象的概念，既可以表示神经网络中的某个层，也可以表示一个包含很多层的神经网络。

在实际使用中，最常见的做法就是继承`nn.Module`，撰写自己的网络/层。一个`nn.Module`实例应该包含一些层以及返回输出的前向传播(forward)方法。

In [5]:
# 使用`nn.Module`实现一个线性模型
from torch import nn

class Linreg(nn.Module):

    def __init__(self, n_features) -> None:
        super(Linreg, self).__init__()
        self.linear = nn.Linear(n_features, 1)
    # forward前向传播
    def forward(self, x):
        y = self.linear(x)
        return y


net = Linreg(num_inputs)
print(net)  # 使用print打印出网络的结构

Linreg(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


还可以使用`nn.Sequential`来更加方便地搭建网络，`Sequential`是一个有序的容器，网络层将按照在传入`Sequential`的顺序依次被添加到计算图中。

In [6]:
# 写法1
net1 = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
)

# 写法2
net2 = nn.Sequential()
net2.add_module('linear', nn.Linear(num_inputs, 1))
# net2.add_module(...)

# 写法3
from collections import OrderedDict
net3 = nn.Sequential(OrderedDict([
        ('linear', nn.Linear(num_inputs, 1))
]))

print(net1)
print(net1[0])

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


可以通过`net.parameters()`来查看模型所有的可学习参数，此参数将返回一个生成器。

In [7]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.4613,  0.5806]], requires_grad=True)
Parameter containing:
tensor([0.1184], requires_grad=True)


> 注意：`torch.nn`仅支持输入一个batch的样本不支持单个样本输入，如果只有单个样本，
> 可使用`input.unsqueeze(0)`来添加一维。

### 3.3.4 初始化模型参数

在使用`net`前，需要初始化模型参数，如线性回归的权重和偏差。PyTorch在`init`模块中提供了多种参数初始化方法。

通过`init.normal_`将权重参数每个元素初始化为随机采样于均值为0，标准差为0.01的正态分布，偏差会初始化为0.

In [9]:
from torch.nn import init

init.normal_(net3[0].weight, mean=0, std=0.01)
init.constant_(net3[0].bias, val=0)

Parameter containing:
tensor([0.], requires_grad=True)

> 注意：对于通过类自定义的神经网络，`net[0].weight`应改为`net.linear.weight`，`bias`亦然。
> 
> 根据下标访问子模块的写法只有当`net`是个`ModuleList`或者`Sequential`实例时才可以。

### 3.3.5 定义损失函数

PyTorch在`nn`模块中提供了各种损失函数，这些损失函数可看作一种特殊的层，Pytorch也将这些损失函数实现为`nn.Module`的子类。

In [10]:
loss = nn.MSELoss()

### 3.3.6 定义优化算法

`torch.optim`模块提供了很多常用的优化算法，比如SGD、Adam和RMSProp等。

下面创建一个用于优化`net`所有参数的优化器实例，并指定学习率为0.03的小批量随机梯度下降(SGD)为优化算法。

In [12]:
import torch.optim as optim

optimizer = optim.SGD(net3.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.03
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


还可以为不同的子网络设置不同的学习率，这在finetune时经常用到。例如：

```python
optimizer = optim.SGD([
            {'param':net.subnet1.parameters()}, # lr=0.03
            {'param':net.subnet2.parameters(), 'lr':0.01}
            ], lr=0.03)
```

### 3.3.7 训练模型

在使用Gluon训练模型时，通过调用`optim`实例的`step`函数来迭代模型参数。按照小批量随机梯度下降的定义，我们在`step`函数中指明批量大小，从而对批量中样本梯度求平均。

In [13]:
epochs = 3
for epoch in range(epochs):
    for X, y in data_iter:
        output = net3(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad()   # 梯度清零
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 0, loss: 0.025421
epoch 1, loss: 0.000057
epoch 2, loss: 0.000057


比较学到的模型参数和真实的模型参数。从`net`获得需要的层，并访问其权重(`weight`)和偏差(`bias`)。

In [14]:
dense = net3[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.9996, -3.4007]], requires_grad=True)
4.2 Parameter containing:
tensor([4.2006], requires_grad=True)
